# 02b - Recordfile Live Preprocessing

In [1]:
import sys
!{sys.executable} -m pip install --upgrade pip -q
!{sys.executable} -m pip install pandas -q
!{sys.executable}  -m pip install numpy -q
!{sys.executable}  -m pip install matplotlib -q
!{sys.executable} -m pip install mgz -q

In [2]:
import numpy as np
import pandas as pd
import json
import os
from mgz import header, body
from mgz.model import parse_match
import pickle
import os
import zipfile


In [3]:
def print_txt(msg: str):
    #print(msg, end = '\r')
    text_file = open("data/scraped_matches/Output_Preprocessing.txt", "w")
    text_file.write(msg)
    text_file.close()

In [4]:
#settings
sizelimit = 999999999999

##directorys
input_path = 'data/scraped_matches/inputs/'
gaia_path = 'data/scraped_matches/gaia_data/'

zip_path = "data/scraped_matches/zipped_recordfiles/"
unzip_path = "data/scraped_matches/unzipped_recordfiles/"

base_directory = "data/scraped_matches/"
input_path = base_directory + "inputs/"


#load parseable recordfiles
with open('data/scraped_matches/unzipper/parseable_fns.pkl', 'rb') as f:
    parseable_fns = pickle.load(f)
    

####limit size for testing
parseable_fns = parseable_fns[:sizelimit]

match_ids = []
remove_fns = []
matches = {}

for fn in parseable_fns:
    match_id = fn.partition("=")[2].partition("&")[0]
    
    if match_id not in match_ids:
        match_ids.append(match_id)
    else:
        remove_fns.append(fn)
    
    #create dict for each match
    matches[match_id] = {}
    matches[match_id]['match_id'] = match_id
    matches[match_id]['fn'] = fn

#remove redundant files
for fn in remove_fns:
    parseable_fns.remove(fn)

print("Matches: " ,len(matches), " - Parseable FNs: ", len(parseable_fns))

Matches:  225398  - Parseable FNs:  225398


**Data Preparation**

In [5]:
#load already prepared matches - to clear reset pkl_matches after
pkl_matches = []

for file in os.listdir(os.fsencode(input_path)):
        filename = os.fsdecode(file)
        match_id = filename[0:-4]
        pkl_matches.append(match_id)

        
#load player masterdata
with open("masterdata/player_elo.pkl", 'rb') as f:
    player_elo = pickle.load(f)
    

print(len(pkl_matches), " found as pkl")


163812  found as pkl


In [ ]:
#read in data from record files


## orga vars
error_count = 0
parsed_files = 0
elo_error = 0
to_be_deleted = []
input_dict = {}
gaia_dict = {}

## content vars for feature creation
masterdata_dict = {}


masterdata_dict['Build'] = []
masterdata_dict['Queue'] = []
masterdata_dict['Research'] = []


for match_index,match_id in enumerate(matches):
    try:
        #unzip and recordfile into unzip folder
        with zipfile.ZipFile(zip_path + "{}".format(matches[match_id]['fn']),"r") as zip_ref:
            zip_ref.extractall(unzip_path)
        
        #create recordfilename
        record_fn = 'AgeIIDE_Replay_{}.aoe2record'.format(match_id)
        
        #parse match data
        with open(unzip_path + record_fn, 'rb') as data:
                eof = os.fstat(data.fileno()).st_size
                body.meta.parse_stream(data)
                while data.tell() < eof:
                    body.operation.parse_stream(data)

        with open(unzip_path + record_fn, 'rb') as data:
                        match = parse_match(data)

        #get general game info  
        matches[match_id]['map'] = match.map.name
        matches[match_id]['map_size'] = match.map.size
        matches[match_id]['duration'] = match.duration.seconds
        matches[match_id]['dataset'] = match.dataset
        matches[match_id]['difficulty'] = match.difficulty
        
        matches[match_id]['input_fn'] = input_path + str(match_id) + ".pkl"
        matches[match_id]['gaia_fn'] = gaia_path + str(match_id) + ".pkl"
        
        #gather player information
        players = match.players
        matches[match_id]['p1_name'] = players[0].name
        matches[match_id]['p2_name'] = players[1].name
        
        #get player ELO from dict, replace with oponent value if missing
        try:
            matches[match_id]['p1_elo'] = player_elo[players[0].profile_id]
            matches[match_id]['p2_elo'] = player_elo[players[1].profile_id]
        except Exception as e:
            try:
                matches[match_id]['p1_elo'] = player_elo[players[0].profile_id]
                matches[match_id]['p2_elo'] = player_elo[players[0].profile_id]
            except:
                matches[match_id]['p1_elo'] = player_elo[players[1].profile_id]
                matches[match_id]['p2_elo'] = player_elo[players[1].profile_id]
            
            
        matches[match_id]['p1_civ'] = players[0].civilization
        matches[match_id]['p2_civ'] = players[1].civilization

        matches[match_id]['p1_xpos'] = players[0].position.x
        matches[match_id]['p2_xpos'] = players[1].position.x
        matches[match_id]['p1_ypos'] = players[0].position.y
        matches[match_id]['p2_ypos'] = players[1].position.y
        
        

        if players[0].winner == True:
            matches[match_id]['winner'] = int(0)
        else:
            matches[match_id]['winner'] = int(1)

        #read inputs and gaia_data if not already in pkl
        if match_id not in pkl_matches: 

            #get all item names
            for object in match.inputs:
                attr_list = object.__dict__.keys()
                break

            if not str(getattr(object,'type')) == "Chat":
                inputs_objects_dict = {}

            for index, object in enumerate(match.inputs):
                inputs_objects_dict[index] = {}


                #create timestamp_seconds
                inputs_objects_dict[index]['ts_seconds'] = getattr(object,'timestamp').seconds

                for attribute in attr_list:
                    inputs_objects_dict[index][attribute] = str(getattr(object,attribute))


                    #create a list of Builds
                    if str(getattr(object,'type')) == 'Build':
                        val = str(getattr(object,'param'))
                        if val not in masterdata_dict['Build'] :
                            masterdata_dict['Build'] .append(val)

                    #create list of possible Units
                    if str(getattr(object,'type')) == 'Queue':
                        val = str(getattr(object,'param'))
                        if val not in masterdata_dict['Queue'] :
                            masterdata_dict['Queue'] .append(val)

                    #create list of Technologies
                    if str(getattr(object,'type')) == 'Research':
                        val = str(getattr(object,'param'))
                        if val not in masterdata_dict['Research'] :
                            masterdata_dict['Research'].append(val)


                    #create redundant single xy coordinates
                    if attribute == 'position':
                        pos = getattr(object,attribute)

                        if pos == None:
                            inputs_objects_dict[index]['x_pos']= None
                            inputs_objects_dict[index]['y_pos'] = None 
                        else:
                            inputs_objects_dict[index]["x_pos"] = getattr(object,attribute).x
                            inputs_objects_dict[index]["y_pos"] = getattr(object,attribute).y

                    #replace playerinfo
                    if attribute == 'player':
                        if inputs_objects_dict[index][attribute] == matches[match_id]['p1_name']:
                            inputs_objects_dict[index][attribute] = 'p1'
                        else:
                            inputs_objects_dict[index][attribute] = 'p2'



            #read gaia data
            gaia_objects_dict = {}
            for object in match.gaia:
                if getattr(object,"name"):
                    #read gaia data in
                    gaia_objects_dict[object.instance_id] = {}
                    gaia_objects_dict[object.instance_id]["name"] = getattr(object,"name")
                    gaia_objects_dict[object.instance_id]["x_pos"] = getattr(object,'position').x
                    gaia_objects_dict[object.instance_id]["y_pos"] = getattr(object,'position').y


            #persistate inputs in pkl file
            output = open(matches[match_id]['input_fn'] , 'wb')
            pickle.dump(inputs_objects_dict, output)
            output.close()

            #persitate gaia_data
            output = open(matches[match_id]['gaia_fn'] , 'wb')
            pickle.dump(gaia_objects_dict, output)
            output.close()
            
            os.remove(unzip_path + record_fn)

    except Exception as e:
        #print(e, "Match_id:", match_id)
        to_be_deleted.append(match_id)
        error_count +=1
        if error_count >= 100:
          pass#break
        pass
 
    parsed_files += 1
    print(f'Parsed {parsed_files}/{len(matches)} - error on: {error_count} - elo error: {elo_error}',  end='\r')

    if match_index % 1000 == 0:
        #print and persistate
        print_txt(f'Parsed: {parsed_files}/{len(matches)} - error on: {error_count} - elo error: {elo_error}')
        #persitate matches
        file_name = base_directory + f'parsed_matches/over_time/{match_index}_matches.pkl'
        output = open(file_name, 'wb')
        pickle.dump(matches, output)
        output.close()
    
    
#errorhandling
print("Parsed:" , parsed_files, "- error on: ", error_count)
#print("Deleting: ", to_be_deleted)
for match_id in to_be_deleted:
    del matches[match_id]

print("Error on ",error_count," matches while parsing recordfile data")
        


In [ ]:
#final persitate matches
file_name = 'data/scraped_matches/parsed_matches/final_matches.pkl'
output = open(file_name, 'wb')
pickle.dump(matches, output)
output.close()

matches_df = pd.DataFrame.from_dict(matches, orient='index')

#final persitate matches df
file_name = 'data/scraped_recordfile_matches.csv'
output = open(file_name, 'wb')
pickle.dump(matches_df, output)
output.close()

matches_df.shape

In [ ]:
#manual add masterdata
masterdata_dict['Ages'] = ['Feudal Age','Castle Age','Imperial Age']

for key in masterdata_dict:
    print(key, ": ",len(masterdata_dict[key]))

#persistate master data
masterdata_path = 'masterdata/'

output = open(masterdata_path + 'masterdata_dict.pkl' , 'wb')
pickle.dump(masterdata_dict, output)
output.close()

masterdata_dict